In [1]:
import cv2
import tensorflow as tf
import numpy as np

print("OpenCV:", cv2.__version__)
print("TensorFlow:", tf.__version__)


OpenCV: 4.11.0
TensorFlow: 2.12.0


# Boom Barrier Open–Close Detection using Computer Vision

This notebook demonstrates an end-to-end computer vision pipeline to classify boom barrier states as **OPEN** or **CLOSED**.


In [2]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split


In [9]:
import os

print("Open images:", len(os.listdir(r"C:\Users\hp pc\Desktop\dataset\open")))
print("Close images:", len(os.listdir(r"C:\Users\hp pc\Desktop\dataset\close")))



Open images: 72
Close images: 72


# Data Preprocessing

In [11]:
dataset_path = r"C:\Users\hp pc\Desktop\dataset"

data = []
labels = []

img_size = 128
classes = ["open", "close"]

for label, category in enumerate(classes):
    folder_path = os.path.join(dataset_path, category)
    for img_name in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img_name)
        img = cv2.imread(img_path)
        img = cv2.resize(img, (img_size, img_size))
        data.append(img)
        labels.append(label)

data = np.array(data) / 255.0
labels = to_categorical(labels)

print("Data shape:", data.shape)
print("Labels shape:", labels.shape)


Data shape: (144, 128, 128, 3)
Labels shape: (144, 2)


In [14]:
#Train test split

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    data, labels, test_size=0.2, random_state=42
)

print("Training samples:", X_train.shape[0])
print("Testing samples:", X_test.shape[0])


Training samples: 115
Testing samples: 29


In [15]:
# Build CNN Model

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(128,128,3)),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(128, activation='relu'),
    Dense(2, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 30, 30, 64)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 57600)             0         
                                                                 
 dense (Dense)               (None, 128)               7

In [17]:
# Train the Model

history = model.fit(
    X_train, y_train,
    epochs=5,
    validation_data=(X_test, y_test)
)


Epoch 1/5
4/4 [==============================] - 6s 1s/step - loss: 2.9961 - accuracy: 0.5304 - val_loss: 0.7214 - val_accuracy: 0.5517
Epoch 2/5
4/4 [==============================] - 4s 1s/step - loss: 0.6811 - accuracy: 0.6522 - val_loss: 0.7014 - val_accuracy: 0.5517
Epoch 3/5
4/4 [==============================] - 4s 1s/step - loss: 0.5157 - accuracy: 0.6870 - val_loss: 0.6052 - val_accuracy: 0.7241
Epoch 4/5
4/4 [==============================] - 4s 1s/step - loss: 0.3875 - accuracy: 0.8174 - val_loss: 0.6558 - val_accuracy: 0.6897
Epoch 5/5
4/4 [==============================] - 4s 1s/step - loss: 0.3049 - accuracy: 0.8609 - val_loss: 0.5208 - val_accuracy: 0.7931


In [18]:
# Evaluate Model

loss, accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy:", accuracy)


1/1 [==============================] - 0s 165ms/step - loss: 0.5208 - accuracy: 0.7931
Test Accuracy: 0.7931034564971924


In [19]:
# Save the Model

model.save("boom_barrier_open_close_model.h5")


In [20]:
# Single Image Prediction (Demo

import numpy as np
import cv2
import os

img = cv2.imread(os.path.join(dataset_path, "open", os.listdir(os.path.join(dataset_path, "open"))[0]))
img = cv2.resize(img, (128,128))
img = np.expand_dims(img, axis=0) / 255.0

prediction = model.predict(img)
result = "OPEN" if np.argmax(prediction)==0 else "CLOSED"

print("Boom Barrier Status:", result)


1/1 [==============================] - 1s 1s/step
Boom Barrier Status: OPEN


## Conclusion
A CNN-based model was built to classify boom barrier states as OPEN or CLOSED.
The pipeline is scalable with more data and can be integrated with CCTV systems.


 - The prediction depends on the input image.

 - When I tested with an open-barrier image, the model correctly predicted OPEN.

 - Testing with a closed-barrier image results in CLOSED prediction, within expected accuracy limits.”

“Although the image belongs to the open class, the model predicted CLOSED with high confidence due to visual features closer to the closed state.
This highlights the probabilistic nature of CNNs and real-world ambiguity.”

## ML Backend Workflow (Boom Barrier Open–Close Detection)

1. **Problem Definition**  
   Defined a binary image classification problem to detect whether a boom barrier is OPEN or CLOSED.

2. **Dataset Collection**  
   Collected a balanced open-source image dataset with two classes:
   - Open barrier images
   - Closed barrier images

3. **Dataset Organization**  
   Organized images using folder-based labeling:
   - dataset/open/
   - dataset/close/

4. **Light EDA (Exploratory Data Analysis)**  
   - Verified image count per class  
   - Visualized sample images  
   - Ensured dataset balance

5. **Data Cleaning (Image-specific)**  
   - Manually removed irrelevant or noisy images  
   - Ensured correct labeling  
   - Kept minor blur/variation to reflect real-world data

6. **Data Preprocessing**  
   - Resized all images to 128×128  
   - Normalized pixel values (0–1 range)  
   - Converted labels to categorical format

7. **Train–Test Split**  
   Split the dataset into training and testing sets (80% train, 20% test).

8. **Model Building**  
   Built a CNN model using:
   - Convolution layers for feature extraction  
   - MaxPooling for dimensionality reduction  
   - Dense layers for classification

9. **Model Training**  
   Trained the CNN model for a limited number of epochs to demonstrate rapid prototyping.

10. **Model Evaluation**  
    Evaluated the model on unseen test data and measured accuracy.

11. **Prediction & Inference**  
    Tested the trained model on single images and analyzed prediction probabilities.

12. **Model Saving**  
    Saved the trained model in `.h5` format for deployment and reuse.

13. **Deployment Readiness**  
    Exposed the trained model via a FastAPI backend to accept image input and return predictions.
